# SAR-LRA VV_VH Models Deployment

This notebook deploys the two models on the composite SAR images acquired in 01_SAR-LRA_Sentinel-1_Image_Acquisition for ascending and descending orbits, VV_VH combination.

In [1]:
import rasterio
print("Rasterio version:", rasterio.__version__) # Version 1.2.10

Rasterio version: 1.2.10


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import imagenet_utils
# from PIL import Image
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from utils import *
from tensorflow.keras.optimizers import Adam
import numpy as np
import time
import cv2
import os

print("TensorFlow version:", tf.__version__)  # Version 2.10.0
print("NumPy version:", np.__version__)       # Version 1.21.5
print("OpenCV version:", cv2.__version__)     # Version 4.5.5

TensorFlow version: 2.10.0
NumPy version: 1.21.5
OpenCV version: 4.5.5


## DESCENDING

### 1. Defining useful variables
Here we define some variables and paths that will be useful to call model weights, images, and saving the outputs.

In [3]:
place = 'Sumatra' # name of the event or location

orbit = 'DESCENDING'

# directory of the SAR composite image
image_path = f'deploy/VV_VH/60_12/SAR_{orbit}_.tif' 

# directory of the weights of the model
weights = f'model_weights/VV_VH_60_nn_noSlope_DESCENDING_60_12_5_size_64_filters_64_batch_size_512_lr_0.001_dropout_0.7_fil1_3_fil2_3_fil3_3.hdf5'

# size of the image
size = 64

# number of bands
channels = 4

### 2. Import Model and Image
Remember to define the model parameters as the one used in the training.

In [4]:
# import model
model = CNN_CBAM(filtersFirstLayer= 64, drop = 0.7, lr = 0.001, input_size = (size, size, channels), loss=focal_loss)
print("[INFO] loading model weights...")
model.load_weights(weights)

# import image
print("[INFO] loading image...")
with rasterio.open(image_path) as ori:
    tmp = np.moveaxis(ori.read(), 0, 2)
orig = np.asarray(tmp)
orig = orig[:,:,:(channels)]

print('MODEL AND IMAGE ARE READY !')

[INFO] loading model weights...
[INFO] loading image...
MODEL AND IMAGE ARE READY !


### 3. Run the Sliding Window algorithm
Here, we execute the sliding window algorithm to extract 64x64 images from the SAR composite image downloaded in the preceding notebook, 01_SAR-LRA_Sentinel-1_Image_Acquisition, with a specified overlap. Subsequently, we store these images along with their corresponding coordinates relative to the original image in two separate lists.

In [5]:
rois = [] # list for images
locs = [] # list for coordinates

ROI_SIZE = (size, size) # 64x64
WIN_STEP = int(size/2)  # 32 to have 50% of overlap - modifiable

start = time.time()

for (x, y, roiOrig) in sliding_window(orig, WIN_STEP, ROI_SIZE):
    w = int(ROI_SIZE[0])
    h = int(ROI_SIZE[1])
    roi = cv2.resize(roiOrig, ROI_SIZE)
    roi = img_to_array(roi)
    rois.append(roi)
    locs.append((x, y, x + w, y + h))
    end = time.time()
print("[INFO] looping over pyramid/windows took {:.5f} seconds".format(end - start))

# convert the ROIs to a NumPy array
rois = np.array(rois)
print('[INFO] You extracted ', len(rois), 'patches')

print('ROIs ARE READY TO BE CLASSIFIED !')

[INFO] looping over pyramid/windows took 11.64422 seconds
[INFO] You extracted  29584 patches
ROIs ARE READY TO BE CLASSIFIED !


### 4. Classify the extracted images
Here, we classify all the images (ROIs) extracted by the sliding window algorithm, saving the predictions as probabilities ranging from 0 to 1, indicating their likelihood of belonging to the landslide class. Subsequently, we will define a probability threshold to determine the class to which they belong.

In [6]:
print("[INFO] classifying ROIs...")
start = time.time()
pred_datagen = ImageDataGenerator()
batch_size = 512
pred_ds = pred_datagen.flow(rois, batch_size = batch_size, seed = 42, shuffle=False)
ynew = model.predict(pred_ds) # predict
end = time.time()
print("[INFO] classifying ROIs took {:.5f} seconds".format(
    end - start))

print('THE MODEL CLASSIFIED ALL THE ROIs !')

[INFO] classifying ROIs...
58/58 [==============================] - 6s 40ms/step
[INFO] classifying ROIs took 5.70280 seconds
THE MODEL CLASSIFIED ALL THE ROIs !


### 5. Count the number of ROIs predicted as landslide

In [15]:
n = 1
for i, prob in enumerate(ynew):
    if prob > 0.6: # probability threshold
        n += 1
print(n)

149


### 6. Append coordinates and probability value of the ROIs predicted as landslide

In [16]:
L = []
P = []
for i, prob in enumerate(ynew):
    if prob > 0.6: # probability threshold
        box = locs[i]
        L.append(box)

### 7. Deploy Non-Maximum Suppression
The non max suppression used is the one developed by Adrian Rosebrock and it is very well explained here:
https://pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/

In [17]:
boxes = np.array(L) # to array
boxes = non_max_suppression_fast(boxes, overlapThresh=0.1) # select overlap threshold between bounding boxes

### 8. Draw the final bounding boxes and save as TIFF file
We delineate the ultimate bounding boxes around the detected landslides by the model, and we generate the final georeferenced raster, marking landslide boxes contours with 1 and non-landslide areas with 0.

In [18]:
# clone the original image
clone = orig.copy() 

# create an empty image (with zeros)
c = np.zeros((clone.shape[0], clone.shape[1]), dtype=np.uint8)

# iterate through the boxes and draw them on the image
for (startX, startY, endX, endY) in boxes:
    # Draw rectangle (bounding box) on the image
    cv2.rectangle(c, (startX, startY), (endX, endY), color=1, thickness=1)

In [19]:
pred_path = f'predictions/{place}_DESCENDING.tif' # directory and name of output TIFF file
ori =  rasterio.open(image_path)
c = np.squeeze(c)

with rasterio.Env():
    profile = ori.profile
    profile.update(
        dtype=rasterio.float32,
        count=1,
        width= c.shape[-1], 
        height= c.shape[-2],
        compress='lzw')
    with rasterio.open(pred_path, 'w', **profile) as dst:
        dst.write(c.astype(rasterio.float32), 1)
        
print('PREDICTION SAVED AS TIFF !')

PREDICTION SAVED !


### 9. Save the predictions as a Shapefile
The predictions are saved in the folder: predictions.

In [20]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# Open the georeferenced TIFF image
with rasterio.open(pred_path) as src:
    # Read the raster data as a numpy array
    image_array = src.read(1)

    # Get the transform (georeferencing information)
    transform = src.transform
    crs = src.crs

    # Generate shapes for areas where pixel values are equal to 1
    shapes = list(shapes(image_array, transform=transform))

# Filter shapes where pixel value is 1
valid_shapes = [s for s, v in shapes if v == 1]

# Convert valid shapes to GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=[shape(s) for s in valid_shapes], crs=crs)
shapefile_path = pred_path + ".shp"
# Save the GeoDataFrame as a shapefile
gdf.to_file(shapefile_path)

## ASCENDING
We will iterate through the methodology once more. The elements that will vary are the model, with the new one trained on Ascending data, and the composite SAR image, which will now be from the Ascending orbit.

### 1. Defining useful variables
Here we define some variables and paths that will be useful to call model weights, images, and saving the outputs.

In [25]:
place = 'Sumatra' # name of the event or location

orbit = 'ASCENDING'

# directory of the SAR composite image
image_path = f'deploy/VV_VH/60_12/SAR_{orbit}_.tif' 

# directory of the weights of the model
weights = f'model_weights/VV_VH_60_nn_noSlope_ASCENDING_60_12_6_size_64_filters_32_batch_size_512_lr_0.001_dropout_0.7_fil1_3_fil2_3_fil3_3.hdf5'

# size of the image
size = 64

# number of bands
channels = 4

### 2. Import Model and Image
Remember to define the model parameters as the one used in the training.

In [26]:
# import model
model = CNN_CBAM(filtersFirstLayer= 32, drop = 0.7, lr = 0.001, input_size = (size, size, channels), loss=focal_loss)
print("[INFO] loading model weights...")
model.load_weights(weights)

# import image
print("[INFO] loading image...")
with rasterio.open(image_path) as ori:
    tmp = np.moveaxis(ori.read(), 0, 2)
orig = np.asarray(tmp)
orig = orig[:,:,:(channels)]

print('MODEL AND IMAGE ARE READY !')

[INFO] loading model weights...
[INFO] loading image...
MODEL AND IMAGE ARE READY !


### 3. Run the Sliding Window algorithm
Here, we execute the sliding window algorithm to extract 64x64 images from the SAR composite image downloaded in the preceding notebook, 01_SAR-LRA_Sentinel-1_Image_Acquisition, with a specified overlap. Subsequently, we store these images along with their corresponding coordinates relative to the original image in two separate lists.

In [27]:
rois = [] # list for images
locs = [] # list for coordinates

ROI_SIZE = (size, size) # 64x64
WIN_STEP = int(size/2)  # 32 to have 50% of overlap - modifiable

start = time.time()

for (x, y, roiOrig) in sliding_window(orig, WIN_STEP, ROI_SIZE):
    w = int(ROI_SIZE[0])
    h = int(ROI_SIZE[1])
    roi = cv2.resize(roiOrig, ROI_SIZE)
    roi = img_to_array(roi)
    rois.append(roi)
    locs.append((x, y, x + w, y + h))
    end = time.time()
print("[INFO] looping over pyramid/windows took {:.5f} seconds".format(end - start))

# convert the ROIs to a NumPy array
rois = np.array(rois)
print('[INFO] You extracted ', len(rois), 'patches')

print('ROIs ARE READY TO BE CLASSIFIED !')

[INFO] looping over pyramid/windows took 8.25361 seconds
[INFO] You extracted  29584 patches
ROIs ARE READY TO BE CLASSIFIED !


### 4. Classify the extracted images
Here, we classify all the images (ROIs) extracted by the sliding window algorithm, saving the predictions as probabilities ranging from 0 to 1, indicating their likelihood of belonging to the landslide class. Subsequently, we will define a probability threshold to determine the class to which they belong.

In [28]:
print("[INFO] classifying ROIs...")
start = time.time()
pred_datagen = ImageDataGenerator()
batch_size = 512
pred_ds = pred_datagen.flow(rois, batch_size = batch_size, seed = 42, shuffle=False)
ynew = model.predict(pred_ds) # predict
end = time.time()
print("[INFO] classifying ROIs took {:.5f} seconds".format(
    end - start))

print('THE MODEL CLASSIFIED ALL THE ROIs !')

[INFO] classifying ROIs...
58/58 [==============================] - 3s 34ms/step
[INFO] classifying ROIs took 2.76405 seconds
THE MODEL CLASSIFIED ALL THE ROIs !


### 5. Count the number of ROIs predicted as landslide

In [29]:
n = 1
for i, prob in enumerate(ynew):
    if prob > 0.6: # probability threshold
        n += 1
print(n)

296


### 6. Append coordinates and probability value of the ROIs predicted as landslide

In [30]:
L = []
P = []
for i, prob in enumerate(ynew):
    if prob > 0.6: # probability threshold
        box = locs[i]
        L.append(box)

### 7. Deploy Non-Maximum Suppression
The non max suppression used is the one developed by Adrian Rosebrock and it is very well explained here:
https://pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/

In [31]:
boxes = np.array(L)
boxes = non_max_suppression_fast(boxes, overlapThresh=0.1)

### 8. Draw the final bounding boxes and save as TIFF file
We delineate the ultimate bounding boxes around the detected landslides by the model, and we generate the final georeferenced raster, marking landslide boxes contours with 1 and non-landslide areas with 0.

In [32]:
# Create an empty image (black image with zeros)
c = np.zeros((clone.shape[0], clone.shape[1]), dtype=np.uint8)

# Iterate through the boxes and draw them on the image
for (startX, startY, endX, endY) in boxes:
    # Draw rectangle (bounding box) on the image
    cv2.rectangle(c, (startX, startY), (endX, endY), color=1, thickness=1)

In [33]:
pred_path = f'predictions/{place}_ASCENDING.tif' # directory and name of output TIFF file
ori =  rasterio.open(image_path)
c = np.squeeze(c)

with rasterio.Env():
    profile = ori.profile
    profile.update(
        dtype=rasterio.float32,
        count=1,
        width= c.shape[-1], 
        height= c.shape[-2],
        compress='lzw')
    with rasterio.open(pred_path, 'w', **profile) as dst:
        dst.write(c.astype(rasterio.float32), 1)
        
print('PREDICTION SAVED AS TIFF !')

PREDICTION SAVED AS TIFF !


### 9. Save the predictions as a Shapefile
The predictions are saved in the folder: predictions.

In [34]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# Open the georeferenced TIFF image
with rasterio.open(pred_path) as src:
    # Read the raster data as a numpy array
    image_array = src.read(1)

    # Get the transform (georeferencing information)
    transform = src.transform
    crs = src.crs

    # Generate shapes for areas where pixel values are equal to 1
    shapes = list(shapes(image_array, transform=transform))

# Filter shapes where pixel value is 1
valid_shapes = [s for s, v in shapes if v == 1]

# Convert valid shapes to GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=[shape(s) for s in valid_shapes], crs=crs)
shapefile_path = pred_path + ".shp"
# Save the GeoDataFrame as a shapefile
gdf.to_file(shapefile_path)